Check the status of tube lines and overground
https://api.tfl.gov.uk/line/mode/tube,overground,dlr,tflrail/status

29-6 added overground state

In [1]:
import requests
import requests_mock
import json
import pprint
import unittest
import sys

In [2]:
def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

In [3]:
# List of allowed lines
lines = ['Bakerloo',
 'Central',
 'Circle',
 'District',
 'DLR',
 'Hammersmith & City',
 'Jubilee',
 'London Overground',
 'Metropolitan',
 'Northern',
 'Piccadilly',
 'TfL Rail',
 'Victoria',
 'Waterloo & City']

In [4]:
ls

Testing 2-7-2017.ipynb  Tube requests.ipynb     test_tube_state.py


In [5]:
len(lines)

14

# Basic function of API

In [6]:
#url = 'https://api.tfl.gov.uk/line/mode/tube/status'   # tube only
url = 'https://api.tfl.gov.uk/line/mode/tube,overground,dlr,tflrail/status'
data = requests.get(url).json() # get the response object
    
print_json(len(data))
print_json(data[0]['name'])
print_json(data[0]['lineStatuses'][0]['statusSeverityDescription'])

14
'Bakerloo'
'Good Service'


In [7]:
print_json(data)

[{'$type': 'Tfl.Api.Presentation.Entities.Line, Tfl.Api.Presentation.Entities',
  'created': '2018-12-11T17:06:05.303Z',
  'crowding': {'$type': 'Tfl.Api.Presentation.Entities.Crowding, '
                        'Tfl.Api.Presentation.Entities'},
  'disruptions': [],
  'id': 'bakerloo',
  'lineStatuses': [{'$type': 'Tfl.Api.Presentation.Entities.LineStatus, '
                             'Tfl.Api.Presentation.Entities',
                    'created': '0001-01-01T00:00:00',
                    'id': 0,
                    'lineId': 'bakerloo',
                    'statusSeverity': 10,
                    'statusSeverityDescription': 'Good Service',
                    'validityPeriods': []}],
  'modeName': 'tube',
  'modified': '2018-12-11T17:06:05.303Z',
  'name': 'Bakerloo',
  'routeSections': [],
  'serviceTypes': [{'$type': 'Tfl.Api.Presentation.Entities.LineServiceTypeInfo, '
                             'Tfl.Api.Presentation.Entities',
                    'name': 'Regular',
       

In [8]:
_lines = [line['name'] for line in data]
_lines

['Bakerloo',
 'Central',
 'Circle',
 'District',
 'DLR',
 'Hammersmith & City',
 'Jubilee',
 'London Overground',
 'Metropolitan',
 'Northern',
 'Piccadilly',
 'TfL Rail',
 'Victoria',
 'Waterloo & City']

In [9]:
data_dict = dict.fromkeys(_lines)    # Create the empty dict for the data
data_dict

{'Bakerloo': None,
 'Central': None,
 'Circle': None,
 'District': None,
 'DLR': None,
 'Hammersmith & City': None,
 'Jubilee': None,
 'London Overground': None,
 'Metropolitan': None,
 'Northern': None,
 'Piccadilly': None,
 'TfL Rail': None,
 'Victoria': None,
 'Waterloo & City': None}

Main logic for parsing the data into a useful dict

In [10]:
for line in data:                     # Assign data via key
    try:
        statuses = [status['statusSeverityDescription'] for status in line['lineStatuses']]
        state = ' + '.join(sorted(set(statuses)))
    
        if state == 'Good Service':   # if good status, this is the only status returned
            reason =  'Nothing to report'
        else:
            reason = ' *** '.join([status['disruption']['additionalInfo'] for status in line['lineStatuses']])

        attr = {'State': state, 'Description': reason}
        data_dict[line['name']] = attr
        
    except Exception as exc:
        line_name = line['name']
        print(f'Exception for {line_name}!')
        print(exc)
    
print_json(data_dict)

{'Bakerloo': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Central': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Circle': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'DLR': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'District': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Hammersmith & City': {'Description': 'Nothing to report',
                        'State': 'Good Service'},
 'Jubilee': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'London Overground': {'Description': 'No service between Edmonton Green and '
                                      'Cheshunt. Use local bus services or '
                                      'services from Ponders End, Brimsdown, '
                                      'Enfield Lock or Waltham Cross, or '
                                      'Greater Anglia services between '
                                      'Liverpool Street and 

## Create function for parsing response data

In [11]:
def parse_api_response(response):
    """Take in the TFL API json."""
    lines = [line['name'] for line in response]    # All available lines
    data_dict = dict.fromkeys(lines)
    
    for line in response:                     # Assign data via key
        try:
            statuses = [status['statusSeverityDescription'] for status in line['lineStatuses']]
            state = ' + '.join(sorted(set(statuses)))
    
            if state == 'Good Service':   # if good status, this is the only status returned
                reason =  'Nothing to report'
            else:
                reason = ' *** '.join([status['disruption']['additionalInfo'] for status in line['lineStatuses']])
                reason = reason.replace('\r\n', ' ')

            attr = {'State': state, 'Description': reason}
            data_dict[line['name']] = attr
        
        except Exception as exc:
            print(exc)
    
    return data_dict

# Create tube class

Wrap in a class with a method for updating and another for returning line of interest. Follow
https://github.com/robmarkcole/home-assistant/blob/dev/homeassistant/components/sensor/bitcoin.py

In [12]:
class TubeData(object):
    """Get the latest tube data from TFL."""
    url = 'https://api.tfl.gov.uk/line/mode/tube,overground,dlr,tflrail/status'
    
    def __init__(self):
        """Initialize the data object."""
        self.data = None

    def update(self):
        """Get the latest data from TFL."""
        response = requests.get(url)
        
        if response.status_code != 200:
            #_LOGGER.warning("Invalid response from API")
            print("Invalid response from API")
        else:
            self.data = parse_api_response(response.json())

In [13]:
test_tube = TubeData()
test_tube.update()
test_tube.data

{'Bakerloo': {'State': 'Good Service', 'Description': 'Nothing to report'},
 'Central': {'State': 'Good Service', 'Description': 'Nothing to report'},
 'Circle': {'State': 'Good Service', 'Description': 'Nothing to report'},
 'District': {'State': 'Good Service', 'Description': 'Nothing to report'},
 'DLR': {'State': 'Good Service', 'Description': 'Nothing to report'},
 'Hammersmith & City': {'State': 'Good Service',
  'Description': 'Nothing to report'},
 'Jubilee': {'State': 'Good Service', 'Description': 'Nothing to report'},
 'London Overground': {'State': 'Part Closure',
  'Description': 'No service between Edmonton Green and Cheshunt. Use local bus services or services from Ponders End, Brimsdown, Enfield Lock or Waltham Cross, or Greater Anglia services between Liverpool Street and Waltham Cross/Cheshunt via Tottenham Hale. No service between Romford and Upminster. Use local London Buses routes 165 248 and 370. No service between Surrey Quays and Clapham Junction, replacement bu

In [14]:
class LondonTubeSensor():    # Entity
    """
    Sensor that reads the status of a tube line.
    """

    ICON = 'mdi:subway'

    def __init__(self, name, data):
        """Initialize the sensor."""
        self._name = name             # the name of the line from the allowed list
        self._data = data
        self._state = None
        self._description = None

    @property
    def name(self):
        """Return the name of the sensor."""
        return self._name

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON

    @property
    def device_state_attributes(self):
        """Return other details about the sensor state."""
        attrs = {}  # {'attribution': 'Data provided by transportapi.com'}
        attrs['Description'] = self._description # if there is data, append
        return attrs

    def update(self):
        """Update the sensor."""
        self._data.update()      # update the data object
        self._state = self._data.data[self.name]['State']
        self._description = self._data.data[self.name]['Description']

In [15]:
data = TubeData()   # init a data object
bakerloo_sensor = LondonTubeSensor('Bakerloo', data)
bakerloo_sensor.update()
print(bakerloo_sensor.state)
print(bakerloo_sensor.device_state_attributes['Description'])

Good Service
Nothing to report


## Unit testing

In [16]:
CONF_LINE = 'line'
config = {CONF_LINE:['Bakerloo', 'London Overground']}

def example_setup_platform(config):
    """Set up the Bitcoin sensors."""


    data = TubeData()
    sensors = []
    for line in config[CONF_LINE]:
        sensors.append(LondonTubeSensor(line, data))

    return sensors

In [17]:
sensors = example_setup_platform(config)

for sensor in sensors:
    sensor.update()
    print(sensor.name)
    print(sensor.state)
    print('*****')

Bakerloo
Good Service
*****
London Overground
Part Closure
*****


In [18]:
valid_response = [
      {
    "id": "london-overground",
    "name": "London Overground",
    "modeName": "overground",
    "disruptions": [
    ],
    "lineStatuses": [
      {
        "statusSeverityDescription": "Minor Delays",

        "disruption": {

          "additionalInfo": "Something\r\nelse"
        }
      }
    ],

  },
]

In [19]:
parse_api_response(valid_response)

{'London Overground': {'State': 'Minor Delays',
  'Description': 'Something else'}}

In [20]:
json.dumps(valid_response)

'[{"id": "london-overground", "name": "London Overground", "modeName": "overground", "disruptions": [], "lineStatuses": [{"statusSeverityDescription": "Minor Delays", "disruption": {"additionalInfo": "Something\\r\\nelse"}}]}]'

In [21]:
class TestLondonTubeSensor(unittest.TestCase):
    
    def test_tube_class(self): 
    
        with requests_mock.mock() as m:
            url = 'https://api.tfl.gov.uk/line/mode/tube,overground,dlr,tflrail/status'
            m.get(url, text=json.dumps(valid_response))
            data = TubeData()
            line = 'London Overground'
            overgound_sensor = LondonTubeSensor(line, data)
            overgound_sensor.update()

            self.assertEqual(overgound_sensor.state, 'Minor Delays')
            self.assertEqual(overgound_sensor.device_state_attributes['Description'], 'Something else')
        
        
unittest.main(argv=['ignored', '-v'], exit=False)

test_tube_class (__main__.TestLondonTubeSensor) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK
